In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from sqlalchemy import create_engine
import time

# Extract

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\alfcu\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [3]:
# Reading billboard.com to extract top 100 list
url = "https://www.billboard.com/charts/hot-100"
browser.visit(url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, "lxml")

In [5]:
big_list = soup.find("ol", {"class": "chart-list__elements"})
billboard = big_list.find_all("li", {"class": "chart-list__element"})

In [6]:
# Building dataframe
ranks = []
songs = []
artists = []

for song in billboard: 
    rank= song.find("span", {"class": "chart-element__rank__number"}).text
    title = song.find("span", {"class": "chart-element__information__song"}).text
    artist = song.find("span", {"class": "chart-element__information__artist"}).text
    
    ranks.append(rank)
    songs.append(title)
    artists.append(artist)
    
df1 = pd.DataFrame()
df1["Rank"] = ranks
df1["Song"] = songs
df1["Artist"] = artists

In [7]:
# Reading apple.com to extract top 100 list
url = "https://music.apple.com/us/playlist/top-100-usa/pl.606afcbb70264d2eb2b51d8dbcfa6a12"
browser.visit(url)

In [8]:
html = browser.html
soup = BeautifulSoup(html, "lxml")

In [9]:
big_list = soup.find("div", {"class": "songs-list"})
apple = big_list.find_all("div", {"class": "songs-list-row"})

In [10]:
# Building dataframe
ranks = []
songs = []
artists = []

for song in apple: 
    rank= song.find("div", {"class": "songs-list-row__rank"}).text
    title = song.find("div", {"class": "songs-list-row__song-name"}).text
    artist = song.find("a", {"class": "songs-list-row__link"}).text
    
    ranks.append(rank)
    songs.append(title)
    artists.append(artist)
    
df2 = pd.DataFrame()
df2["Rank"] = ranks
df2["Song"] = songs
df2["Artist"] = artists

In [11]:
# Reading rollingstone.com to extract top 100 list
url = "https://www.rollingstone.com/charts/songs/"
browser.visit(url)

In [12]:
# Clicking 'Load more' button on the site to expand the list - initial view only 15 songs
for i in range(6):
     browser.find_by_text('Load more').first.click()
     time.sleep(1)

In [13]:
html = browser.html
soup = BeautifulSoup(html, "lxml")

In [14]:
big_list = soup.find("section", {"class": "l-section__charts"})
listings = soup.find_all("div", {"class": "c-chart__table--top"})

In [15]:
# Building dataframe
ranks = []
titles = []
captions = []

for listing in listings:
    rank = listing.find("div", {"class": "c-chart__table--rank"}).text
    title = listing.find("div", {"class": "c-chart__table--title"}).text
    caption = listing.find("div", {"class": "c-chart__table--caption"}).text
    
    ranks.append(rank)
    titles.append(title)
    captions.append(caption)
    

df3 = pd.DataFrame()
df3["Rank"] = ranks
df3["Song"] = titles
df3["Artist"] = captions

# Transform

In [16]:
# Adding website info to dataframe in source column
df1["Source"] = 'https://www.billboard.com'

In [17]:
# Building source lookup table
source = [{'website':'https://www.billboard.com'}]
df_source = pd.DataFrame(source)

In [18]:
# Adding website info to dataframe in source column
df2["Source"] = 'https://music.apple.com'

In [19]:
# Adding second source to lookup table
source2 = [{'website':'https://music.apple.com'}]
df_source2 = pd.DataFrame(source2)
df_source = df_source.append(df_source2, ignore_index = True)

In [20]:
# Adding website info to dataframe in source column
df3["Source"] = "https://www.rollingstone.com"

In [21]:
# Adding third source to lookup table
source3 = [{'website':'https://www.rollingstone.com'}]
df_source3 = pd.DataFrame(source3)
df_source = df_source.append(df_source3, ignore_index = True)

In [22]:
# Three top 100 lists combined together using concat to form a dataframe of 300 rows
df_final = pd.concat([df1, df2, df3], ignore_index = True)

In [23]:
# Changing column name to lowercase so artist lookup table can be accepted during the load process
df_artist = pd.DataFrame(df_final["Artist"].unique())
df_artist.rename(columns = {0:"artist"}, inplace = True)

# Load

In [24]:
# Connecting to cloud database
rds_connection_string = "postgres:password@34.71.136.159:5432/postgres"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [25]:
engine.table_names()

<ipython-input-25-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['artist', 'main', 'source']

In [26]:
# Loading artist lookup table to database
df_artist.to_sql(name='artist', con=engine, if_exists='append', index=False, method="multi")

In [27]:
# Creating dataframe based on lookup table in database
df_artist_sql = pd.read_sql_query('select artist_id, artist from artist', con=engine)

In [28]:
# Loading source lookup table to database
df_source.to_sql(name='source', con=engine, if_exists='append', index=False, method="multi")

In [29]:
# Creating dataframe based on lookup table in database
df_source_sql = pd.read_sql_query('select source_id, website from source', con=engine)

In [30]:
# Merging master dataframe with artist lookup to assign artist IDs
df_final2 = df_final.merge(df_artist_sql, left_on='Artist', right_on='artist', how='left')
df_final2.drop(["Artist", "artist"], axis=1, inplace=True)

In [31]:
# Merging updated master dataframe with source lookup to assign source IDs
df_final3 = df_final2.merge(df_source_sql, left_on='Source', right_on='website', how='left')
df_final3.drop(["Source", "website"], axis=1, inplace=True)

In [32]:
# Setting column names to lowercase for master table to be accepted
df_final3.columns = df_final3.columns.str.lower()

In [33]:
# Loading master table to database
df_final3.to_sql(name='main', con=engine, if_exists='append', index=False, method="multi")

In [34]:
# View of the master table in the database
pd.read_sql_query('select * from main', con=engine).head()

,id,rank,song,artist_id,source_id,last_updated
0,1,1,Way 2 Sexy,1,1,2021-09-14 19:08:54.360802
1,2,2,Girls Want Girls,2,1,2021-09-14 19:08:54.360802
2,3,3,Fair Trade,3,1,2021-09-14 19:08:54.360802
3,4,4,Champagne Poetry,4,1,2021-09-14 19:08:54.360802
4,5,5,Knife Talk,5,1,2021-09-14 19:08:54.360802
